<a href="https://colab.research.google.com/github/tiwaripari/ML/blob/main/FinalMusicFeatureExtracted.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

import librosa
import librosa.display
import IPython.display as ipd
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from scipy.stats import kurtosis, skew
from scipy.signal import find_peaks

In [6]:
def feature_extraction(file_path):
  # Load the audio file
  x, sample_rate = librosa.load(file_path)
  features = {}

  mfcc = []
  mfcc = np.mean(librosa.feature.mfcc(y=x, sr=sample_rate, n_mfcc=13).T, axis=0)
  for i in range(1, 14):
    features[f'_MFCC_Mean_{i}'] = mfcc[i-1].mean()

  chroma = np.mean(librosa.feature.chroma_stft(y=x, sr=sample_rate).T,axis=0)
  for i in range(1, 13):
        features[f'_Chromagram_Mean_{i}'] = chroma[i-1]

  spectrogram = librosa.stft(x)
  power_spectrogram = np.abs(spectrogram) ** 2

  features['_HarmonicChangeDetectionFunction_Mean'] = librosa.feature.delta(power_spectrogram, order=1, axis=-1).mean()
  features['_HarmonicChangeDetectionFunction_Std'] = librosa.feature.delta(power_spectrogram, order=1,axis=-1).std()
  features['_HarmonicChangeDetectionFunction_Slope'] = np.mean(librosa.feature.delta(power_spectrogram, order=1, axis=-1))

  # Apply Harmonic/Percussive Source Separation (HPSS)
  harmonic, percussive = librosa.effects.hpss(y=x)

  # Calculate the autocorrelation of the harmonic component
  autocorr = np.correlate(harmonic, harmonic, mode='full')

  # Find the peak in the autocorrelation function
  lags = np.arange(-len(harmonic) + 1, len(harmonic))
  positive_lags = lags[lags >= 0]
  periodicity = autocorr[positive_lags]

  # Calculate the HCDF-like feature values
  features['_HarmonicChangeDetectionFunction_PeriodFreq'] = sample_rate / (positive_lags[np.argmax(periodicity)] + 1)
  features['_HarmonicChangeDetectionFunction_PeriodAmp'] = np.max(periodicity)
  features['_HarmonicChangeDetectionFunction_PeriodEntropy'] = -np.sum(periodicity * np.log2(periodicity)) / np.sum(periodicity)


  features['_RMSenergy_Mean'] = librosa.feature.rms(y=x).mean()
  features['_Lowenergy_Mean'] = librosa.feature.spectral_contrast(S=power_spectrogram).mean()
  features['_Fluctuation_Mean'] = librosa.feature.spectral_flatness(S=power_spectrogram).mean()
  features['_Tempo_Mean'] = librosa.feature.tempo(y=x, sr=sample_rate)[0].mean()

  # Compute the spectral centroid feature
  spectral_centroids = librosa.feature.spectral_centroid(y=x, sr=sample_rate)
  features['_Spectralcentroid_Mean'] = spectral_centroids.mean()

# Compute the spectral spread feature
  spectral_spreads = librosa.feature.spectral_bandwidth(y=x, sr=sample_rate)
  features['_Spectralspread_Mean'] = spectral_spreads.mean()

# Calculate the magnitude spectrum
  magnitude_spectrum = np.abs(np.fft.fft(x))

# Calculate the spectral kurtosis
  spectral_kurtosis = kurtosis(magnitude_spectrum)
  features['_Spectralkurtosis_Mean'] = np.mean(spectral_kurtosis)

# Calculate the spectral skewness
  spectral_skewness = skew(magnitude_spectrum)
  features['_Spectralskewness_Mean'] = np.mean(spectral_skewness)

# Compute the spectral flatness feature
  spectral_flatness = librosa.feature.spectral_flatness(y=x)
  features['_Spectralflatness_Mean'] = spectral_flatness.mean()

# Compute the entropy of spectrum feature
  entropy_of_spectrum = librosa.feature.spectral_contrast(y=x, sr=sample_rate)
  features['_EntropyofSpectrum_Mean'] = entropy_of_spectrum.mean()

  # Calculate the magnitude spectrogram
  S = np.abs(librosa.stft(y=x))

# Calculate the roughness feature
  roughness = librosa.feature.spectral_rolloff(S=S, sr=sample_rate)
  features['_Roughness_Mean'] = np.mean(roughness)

# Calculate the roughness slope feature
  roughness_slope = np.diff(roughness, axis=1)
  features['_Roughness_Slope'] = np.mean(roughness_slope)

# Calculate the zero-crossing rate feature
  zero_crossing_rate = librosa.feature.zero_crossing_rate(y=x)
  features['_Zero-crossingrate_Mean'] = np.mean(zero_crossing_rate)

# Calculate the attack time feature
  attack_time = librosa.onset.onset_strength(y=x, sr=sample_rate)
  features['_AttackTime_Mean'] = np.mean(attack_time)

# Calculate the attack time slope feature
  attack_time_slope = np.diff(attack_time)
  features['_AttackTime_Slope'] = np.mean(attack_time_slope)

# Calculate the rolloff feature
  rolloff = librosa.feature.spectral_rolloff(y=x, sr=sample_rate)
  features['_Rolloff_Mean'] = np.mean(rolloff)

# Calculate the event density feature
  event_density = np.sum(magnitude_spectrum > np.max(magnitude_spectrum) * 0.5)
  features['_Eventdensity_Mean'] = event_density / len(magnitude_spectrum)

# Calculate the pulse clarity feature
  peaks, _ = find_peaks(x)
  features['_Pulseclarity_Mean'] = np.mean(peaks)

# Calculate the brightness feature
  brightness = librosa.feature.spectral_flatness(y=x)
  features['_Brightness_Mean'] = np.mean(brightness)

  return features

In [7]:
x = '/content/drive/MyDrive/TunePocket-Rock-Party-30-Sec-Intro-Preview.mp3'
features = {}
features = feature_extraction(x)

<ipython-input-6-204c960994cf>:36: RuntimeWarning: invalid value encountered in log2
  features['_HarmonicChangeDetectionFunction_PeriodEntropy'] = -np.sum(periodicity * np.log2(periodicity)) / np.sum(periodicity)


In [8]:
features

{'_MFCC_Mean_1': -22.972054,
 '_MFCC_Mean_2': 84.601036,
 '_MFCC_Mean_3': -28.321905,
 '_MFCC_Mean_4': 32.63283,
 '_MFCC_Mean_5': -1.4422629,
 '_MFCC_Mean_6': 5.140584,
 '_MFCC_Mean_7': 1.2359622,
 '_MFCC_Mean_8': 6.829946,
 '_MFCC_Mean_9': 1.3748925,
 '_MFCC_Mean_10': 7.0407195,
 '_MFCC_Mean_11': -1.3476068,
 '_MFCC_Mean_12': 5.826765,
 '_MFCC_Mean_13': -1.3939185,
 '_Chromagram_Mean_1': 0.45105198,
 '_Chromagram_Mean_2': 0.40799463,
 '_Chromagram_Mean_3': 0.49941653,
 '_Chromagram_Mean_4': 0.77754074,
 '_Chromagram_Mean_5': 0.49573925,
 '_Chromagram_Mean_6': 0.40501738,
 '_Chromagram_Mean_7': 0.4624765,
 '_Chromagram_Mean_8': 0.57066023,
 '_Chromagram_Mean_9': 0.74407774,
 '_Chromagram_Mean_10': 0.5218353,
 '_Chromagram_Mean_11': 0.39746425,
 '_Chromagram_Mean_12': 0.34780678,
 '_HarmonicChangeDetectionFunction_Mean': -0.011699124,
 '_HarmonicChangeDetectionFunction_Std': 75.90025,
 '_HarmonicChangeDetectionFunction_Slope': -0.011699124,
 '_HarmonicChangeDetectionFunction_PeriodFreq'

In [9]:
len(features)

50